In [1]:
import cv2
import math
import numpy as np
from scipy.linalg import svd
import tensorflow as tf           #2~3 Tensor Computing Libraries
import tensorly as tl             #4~5 Tensor expansion and decomposition
from tensorly.decomposition import parafac
import scipy.io as sio
from sklearn import *
import warnings
import sys
from numpy import *
import copy
import time
import sklearn
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import cohen_kappa_score
import heapq
import openpyxl
from datetime import datetime

np.set_printoptions(threshold=np.inf)
DIR_NUM = 16   #Number of label types
n3 = 3         #The dimension of the third dimension of the tensor
train_num = 11 #Number of trainings
test_num = 10 #Number of tests
train_path="C:\\Users\\Administrator\\Desktop\\Temporary\\ETH-80\\fenlei\\40\\train5\\%d.png"            #Training set path
test_path="C:\\Users\\Administrator\\Desktop\\Temporary\\ETH-80\\fenlei\\40\\test\\%d.png"
#trainlable="C:\\Users\\Administrator\\Desktop\\Color FERET\\fenlei\\train.txt"  #Read training sample labels
#testlable="C:\\Users\\Administrator\\Desktop\\Color FERET\\fenlei\\test.txt"#Read testing sample labels
Imagesize=np.array([100,100])
TrnData=np.zeros([Imagesize[0],Imagesize[1],n3,train_num*DIR_NUM],np.float64)
TrnLabels=np.zeros(DIR_NUM*train_num,np.uint8)
TestData=np.zeros((Imagesize[0],Imagesize[1],n3,test_num*DIR_NUM),np.float64)
TestLabels=np.zeros(DIR_NUM*test_num,np.uint8)
AvgData=np.zeros((Imagesize[0],Imagesize[1],n3),np.float64)
#Import training data

cnt=0
for j in range(train_num*DIR_NUM):
    im=cv2.imread(train_path%(j),1)# File Path
    norm_im = np.zeros(im.shape, dtype=np.float32)
    cv2.normalize(im, norm_im, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    TrnData[:,:,:,j]=norm_im
    AvgData+=norm_im
AvgData=AvgData/TrnData.shape[3]
#Import testing data
for j in range(test_num*DIR_NUM):
    im=cv2.imread(test_path%(j),1)
    norm_im = np.zeros(im.shape, dtype=np.float32)
    cv2.normalize(im, norm_im, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    TestData[:,:,:,j]=norm_im
# Add labels to data
ImgIdx=range(DIR_NUM)
TrnLabels = kron(ImgIdx, ones((1, train_num))[0])
TestLabels = kron(ImgIdx, ones((1, test_num))[0])
#TrnLabels=np.loadtxt(trainlable)
#TestLabels=np.loadtxt(testlable)

'''
#Import training data
for j in range(1,DIR_NUM+1):
    #random.shuffle(s)
    for i in range(1,train_num+1):
        im=cv2.imread(train_path%(j,i-1),1
        #im_temp1=tl.unfold(im,mode=1)# Normalized
        #im = (im) / (np.linalg.norm(im_temp1,ord='fro') )
        norm_im = np.zeros(im.shape, dtype=np.float32)
        cv2.normalize(im, norm_im, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        TrnData[:,:,:,(i-1)+(j-1)*train_num]=norm_im
        AvgData+=norm_im
        TrnLabels[i+(j-1)*train_num-1]=j

    for i in range(1,test_num+1):
        im=cv2.imread(test_path%(j,i-1),1)
        #im_temp1=tl.unfold(im,mode=1)# Normalized
        #im = (im) / (np.linalg.norm(im_temp1,ord='fro') )
        norm_im = np.zeros(im.shape, dtype=np.float32)
        cv2.normalize(im, norm_im, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        TestData[:,:,:,i+(j-1)*test_num-1]=norm_im
        TestLabels[i+(j-1)*test_num-1]=j
AvgData=AvgData/TrnData.shape[3]
'''

#主程序
if __name__=='__main__':
    warnings.filterwarnings("ignore")
    print('\n ====== Sample centralization ======= \n')
    Scale_size = [2**(-3),2**(-2),2**(-1),1,2**(1),2**(2),2**(3)]#Scaling parameter size
    for Seacrh in range(len(Scale_size)):
        OMT_A=AvgData
        print('\n ====== Parameter Definition ======= \n')
        Anchor_Num=3
        mr=Anchor_Num
        start_time = time.time()
        Dim = 3
        kDim1 =Anchor_Num
        kDim2 =Anchor_Num
        rank = [kDim1,kDim2,Dim]#Dimension of the core tensor
        Dir_nums = DIR_NUM
        train_Nums = train_num
        PROJ_U1 = np.ones((Imagesize[0], kDim1))#Define the projection matrix
        PROJ_U2 = np.ones((Imagesize[1], kDim2))
        PROJ_U3 = np.ones((Dim, Dim))
        
        # Define the vector lengths and beta
        vector_length = Dir_nums*train_Nums
        # Initialize the variables
        Singel_Rec = np.ones((vector_length,1))
        pi_t = np.ones((vector_length, 1))
        si_t = np.ones((vector_length, 1))
        max_iter=10#Maximum number of iterations
        Obj_Rec = np.ones((max_iter, 1))
        Ker_scale=Scale_size[Seacrh]
        print('\n ====== Find eigenvector ======= \n')
        # Total dispersion matrix for each dimension
        for d in range(1, 4):
            if d == 1:
                Covar = np.zeros((Imagesize[0], Imagesize[0]))
                PROJ_U = PROJ_U1
            elif d == 2:
                Covar = np.zeros((Imagesize[1], Imagesize[1]))
                PROJ_U = PROJ_U2
            elif d == 3:
                Covar = np.zeros((n3, n3))
                PROJ_U = PROJ_U3

            for i in range(Dir_nums * train_Nums):
                OMT_X = TrnData[:, :, :, i]-OMT_A
                OMT_Xd=tl.unfold(OMT_X,mode=d-1)# on the tensor along n-module expansions
                Covar += np.matmul(OMT_Xd, OMT_Xd.T)

            # Sorted eigenvalues and corresponding eigenvectors
            if d == 1:
                D1, E1 = linalg.eig(mat(Covar))
                V1 = E1[:,0:kDim1]
                V1 = asarray(V1)
                PROJ_U1 = V1
            elif d == 2:
                D1, E1 = linalg.eig(mat(Covar))
                V1 = E1[:,0:kDim2]
                V1 = asarray(V1)
                PROJ_U2 = V1
            elif d == 3:
                D1, E1 = linalg.eig(mat(Covar))
                V1 = E1[:,0:Dim]
                V1 = asarray(V1)
                PROJ_U3 = V1
            
        #Construct the m-mode projection matrix
        Mode_U=[]
        Mode_U.append(PROJ_U1)
        Mode_U.append(PROJ_U2)
        Mode_U.append(PROJ_U3)
        # Loop over i
        for i in range(Dir_nums*train_Nums):
            OMT_X=TrnData[:,:,:,i]- OMT_A
            Core_tensor = tl.tenalg.multi_mode_dot(OMT_X,Mode_U,list(range(len(rank))),transpose=True)# Find the core tensor
            Temp_tensor = tl.tenalg.multi_mode_dot(Core_tensor,Mode_U,list(range(len(rank))),transpose=False)# Seek the reconstruction tensor
            bt_temp=OMT_X-Temp_tensor
            bt7=tl.unfold(bt_temp,mode=1)# on the tensor along the 1-module expansion
            bt8=tl.unfold(OMT_X,mode=1)
            Singel_Rec[i] = (np.linalg.norm(bt7,ord='fro')**2)
        
        # Find the interquartile range IQR
        q1 = np.percentile(Singel_Rec, 25)
        q3 = np.percentile(Singel_Rec, 75)
        # Calculate the interquartile range (IQR)
        iqr = q3-q1            
        # Compare squares to array element sizes
        squaredData = iqr/1.354
        min_value = np.std(Singel_Rec)
        minimumValue = min(min_value, squaredData)
        # Output minimum
        Kernel_size = 1.06*minimumValue*(Dir_nums*train_Nums)**(-0.2)
        Ker_beta=Kernel_size
        
        # Calculate the value of the objective function
        Obj_value = 0
        for i in range(Dir_nums*train_Nums):
            Obj_value +=math.exp(-Ker_scale*(Singel_Rec[i])/(2 * Ker_beta))
        #print('Objective value:  %.2f%%'%Obj_value)
        print(f"Objective value: {Obj_value}")
        print('\n ====== Iterative updating of the solution ======= \n')
        t=0# Optimally solving for cyclic variables
        Thresholds = 0.00001   # Set a value as a threshold
        while True:
            for i in range(Dir_nums*train_Nums):
                pi_t[i] = -Ker_scale/(2 * Ker_beta)*math.exp(-Ker_scale*(Singel_Rec[i])/(2 * Ker_beta))
                si_t[i] = -pi_t[i]
            
            # Update the optimal projection tensor
            OMT_Atemp1 = np.zeros((Imagesize[0], Imagesize[1], n3))
            OMT_Atemp2 = 0
            for i in range(Dir_nums*train_Nums):
                OMT_Atemp1 += si_t[i] * TrnData[:, :, :, i]
                OMT_Atemp2 += si_t[i]
            OMT_A = OMT_Atemp1 / OMT_Atemp2
            
            # Total dispersion matrix for each dimension
            for d in range(1, 4):
                if d == 1:
                    Covar_A = np.zeros((Imagesize[0], Imagesize[0]))
                    PROJ_U = PROJ_U1
                elif d == 2:
                    Covar_A = np.zeros((Imagesize[1], Imagesize[1]))
                    PROJ_U = PROJ_U2
                elif d == 3:
                    Covar_A = np.zeros((n3, n3))
                    PROJ_U = PROJ_U3

                for i in range(Dir_nums * train_Nums):
                    OMT_X = TrnData[:, :, :, i]- OMT_A
                    #Core_tensor = tl.tenalg.multi_mode_dot(OMT_X,Mode_U,list(range(len(rank))),transpose=True)
                    Temp_tensorn = tl.tenalg.multi_mode_dot(OMT_X,Mode_U,skip=d-1, modes=list(range(len(rank))),transpose=True)#求-n张量
                    OMT_Xd=tl.unfold(Temp_tensorn,mode=d-1)
                    Covar_A =Covar_A + np.matmul(OMT_Xd, OMT_Xd.T)*si_t[i]
                Covar_A[np.isnan(Covar_A)] = 0
                # Sorted eigenvalues and corresponding eigenvectors
                if d == 1:
                    Covar=np.matmul(Covar_A,PROJ_U)
                    U, S, V = svd(Covar)
                    I_U = np.eye(Imagesize[0], kDim1)
                    PROJ_U1 = np.dot(np.dot(U, I_U), V.T)
                    Mode_U[d-1] = PROJ_U1
                elif d == 2:
                    Covar=np.matmul(Covar_A,PROJ_U)
                    U, S, V = svd(Covar)
                    I_U = np.eye(Imagesize[0], kDim2)
                    PROJ_U2 = np.dot(np.dot(U, I_U), V.T)
                    Mode_U[d-1] = PROJ_U2
                elif d == 3:
                    Covar=np.matmul(Covar_A,PROJ_U)
                    U, S, V = svd(Covar)
                    I_U = np.eye(n3, Dim)
                    PROJ_U3 = np.dot(np.dot(U, I_U), V.T)
                    Mode_U[d-1] = PROJ_U3

            # Loop over i
            for i in range(Dir_nums*train_Nums):
                OMT_X=TrnData[:,:,:,i]- OMT_A
                Core_tensor = tl.tenalg.multi_mode_dot(OMT_X,Mode_U,list(range(len(rank))),transpose=True)
                Temp_tensor = tl.tenalg.multi_mode_dot(Core_tensor,Mode_U,list(range(len(rank))),transpose=False)
                bt_temp=OMT_X-Temp_tensor
                bt7=tl.unfold(bt_temp,mode=1)
                bt8=tl.unfold(OMT_X,mode=1)
                Singel_Rec[i] = (np.linalg.norm(bt7,ord='fro')**2)
            
            #Updating the kernel width
            q1 = np.percentile(Singel_Rec, 25)
            q3 = np.percentile(Singel_Rec, 75)
            
            iqr = q3-q1            
            
            squaredData = iqr/1.354
            min_value = np.std(Singel_Rec)
            minimumValue = min(min_value, squaredData)
       
            Kernel_size = 1.06*minimumValue*(Dir_nums*train_Nums)**(-0.2)
            Ker_beta=Kernel_size
            
            
            Obj_value = 0
            for i in range(Dir_nums*train_Nums):
                Obj_value += math.exp(-Ker_scale*(Singel_Rec[i])/(2 * Ker_beta)) 
            Obj_Rec[t] = Obj_value
            print(f"Objective value: {Obj_Rec[t]}")
            t=t+1
            if t>=2:
                temp_t=abs(Obj_Rec[t-1]-Obj_Rec[t-2])/abs(Obj_Rec[t-1])
                # Check for loop termination conditions
                if temp_t < Thresholds or t == max_iter:
                    break
        
        # The program runtime is
        end_time = time.time()
        execution_time1 = end_time - start_time
        print("The program runtime is", execution_time1, "秒")
        
        print("Update the parameters to complete and perform the projection matrix solution")
        for Exper_Num in range(15,100,15):
            start_time = time.time()
            Dim = 3
            kDim1 =Exper_Num
            kDim2 =Exper_Num
            rank = [kDim1,kDim2,Dim]
            Dir_nums = DIR_NUM
            train_Nums = train_num
            PROJ_U1 = np.ones((Imagesize[0], kDim1))
            PROJ_U2 = np.ones((Imagesize[1], kDim2))
            PROJ_U3 = np.ones((Dim, Dim))

            # Define the vector lengths and beta
            vector_length = Dir_nums*train_Nums
            # Initialize the variables
            Singel_Rec = np.ones((vector_length,1))
            Singel_Obj = np.ones((vector_length,1))
            max_iter=10
            Obj_Rec = np.ones((max_iter+1, 1))
            Accrucy_Rec = np.ones((max_iter, 1))
            print('\n ====== Find eigenvector ======= \n')

            for d in range(1, 4):
                if d == 1:
                    Covar = np.zeros((Imagesize[0], Imagesize[0]))
                    PROJ_U = PROJ_U1
                elif d == 2:
                    Covar = np.zeros((Imagesize[1], Imagesize[1]))
                    PROJ_U = PROJ_U2
                elif d == 3:
                    Covar = np.zeros((n3, n3))
                    PROJ_U = PROJ_U3

                for i in range(Dir_nums * train_Nums):
                    OMT_X = TrnData[:, :, :, i]-OMT_A
                    OMT_Xd=tl.unfold(OMT_X,mode=d-1)
                    Covar += np.matmul(OMT_Xd, OMT_Xd.T)
                Covar[np.isnan(Covar)] = 0
                # Sorted eigenvalues and corresponding eigenvectors
                if d == 1:
                    D1, E1 = linalg.eig(mat(Covar))
                    V1 = E1[:,0:kDim1]
                    V1 = asarray(V1)
                    PROJ_U1 = V1
                elif d == 2:
                    D1, E1 = linalg.eig(mat(Covar))
                    V1 = E1[:,0:kDim2]
                    V1 = asarray(V1)
                    PROJ_U2 = V1
                elif d == 3:
                    D1, E1 = linalg.eig(mat(Covar))
                    V1 = E1[:,0:Dim]
                    V1 = asarray(V1)
                    PROJ_U3 = V1
            
            Mode_U=[]
            Mode_U.append(PROJ_U1)
            Mode_U.append(PROJ_U2)
            Mode_U.append(PROJ_U3)
            # Loop over i
            for i in range(Dir_nums*train_Nums):
                b=TrnData[:,:,:,i]- OMT_A
                Core_tensor = tl.tenalg.multi_mode_dot(b,Mode_U,list(range(len(rank))),transpose=True)
                Temp_tensor = tl.tenalg.multi_mode_dot(Core_tensor,Mode_U,list(range(len(rank))),transpose=False)
                bt_temp=b-Temp_tensor
                bt7=tl.unfold(bt_temp,mode=1)
                Singel_Rec[i] = (np.linalg.norm(bt7,ord='fro')** 2)


            Obj_value = 0
            t=0
            for i in range(Dir_nums*train_Nums):
                Obj_value += math.exp(-Ker_scale*(Singel_Rec[i])/(2 * Ker_beta))
            Obj_Rec[t] = Obj_value
            print('Objective value:  %.2f%%'%Obj_value)
            print('\n ====== Iterative updating of the solution ======= \n')
            Thresholds = 0.00001         
            while True:
                
                for d in range(1, 4):
                    if d == 1:
                        Covar_A = np.zeros((Imagesize[0], Imagesize[0]))
                        PROJ_U = PROJ_U1
                    elif d == 2:
                        Covar_A = np.zeros((Imagesize[1], Imagesize[1]))
                        PROJ_U = PROJ_U2
                    elif d == 3:
                        Covar_A = np.zeros((n3, n3))
                        PROJ_U = PROJ_U3

                    for i in range(Dir_nums * train_Nums):
                        OMT_X = TrnData[:, :, :, i]- OMT_A
                        #Core_tensor = tl.tenalg.multi_mode_dot(OMT_X,Mode_U,list(range(len(rank))),transpose=True)
                        Temp_tensorn = tl.tenalg.multi_mode_dot(OMT_X,Mode_U,skip=d-1, modes=list(range(len(rank))),transpose=True)#求-n张量
                        OMT_Xd=tl.unfold(Temp_tensorn,mode=d-1)
                        Covar_A = Covar_A+np.matmul(OMT_Xd, OMT_Xd.T)*si_t[i]
                    Covar_A[np.isnan(Covar_A)] = 0
                    # Sorted eigenvalues and corresponding eigenvectors
                    if d == 1:
                        Covar=np.matmul(Covar_A,PROJ_U)
                        U, S, V = svd(Covar)
                        I_U = np.eye(Imagesize[0], kDim1)
                        PROJ_U1 = np.dot(np.dot(U, I_U), V.T)
                        Mode_U[d-1] = PROJ_U1
                    elif d == 2:
                        Covar=np.matmul(Covar_A,PROJ_U)
                        U, S, V = svd(Covar)
                        I_U = np.eye(Imagesize[0], kDim2)
                        PROJ_U2 = np.dot(np.dot(U, I_U), V.T)
                        Mode_U[d-1] = PROJ_U2
                    elif d == 3:
                        Covar=np.matmul(Covar_A,PROJ_U)
                        U, S, V = svd(Covar)
                        I_U = np.eye(n3, Dim)
                        PROJ_U3 = np.dot(np.dot(U, I_U), V.T)
                        Mode_U[d-1] = PROJ_U3

                # Loop over i
                Rec_errors=0
                for i in range(Dir_nums*train_Nums):
                    b=TrnData[:,:,:,i]- OMT_A
                    Core_tensor = tl.tenalg.multi_mode_dot(b,Mode_U,list(range(len(rank))),transpose=True)
                    Temp_tensor = tl.tenalg.multi_mode_dot(Core_tensor,Mode_U,list(range(len(rank))),transpose=False)
                    bt_temp=b-Temp_tensor
                    bt7=tl.unfold(bt_temp,mode=1)
                    Singel_Rec[i] = (np.linalg.norm(bt7,ord='fro')** 2)

                # Calculate the value of the objective function
                Obj_value = 0
                for i in range(Dir_nums*train_Nums):
                    Obj_value += math.exp(-Ker_scale*(Singel_Rec[i])/(2 * Ker_beta))
                t=t+1
                # Output and save objective function values
                Obj_Rec[t] = Obj_value
                print('Objective value:  %.2f%%'%Obj_Rec[t])
                temp_t=abs(Obj_Rec[t]-Obj_Rec[t-1])/abs(Obj_Rec[t])
                # Check for loop termination conditions
                if temp_t < Thresholds or t == max_iter:
                    break

            # The program runtime is
            end_time = time.time()
            execution_time = execution_time1+end_time - start_time
            print("The program run time is:", execution_time, "秒")
            
            print('\n ====== Extract features ======= \n')
            
            ftr=np.zeros([TrnData.shape[3],kDim1*kDim2*Dim],np.float32)# Define the feature matrix after dimensionality reduction
            fte=np.zeros([TestData.shape[3],kDim1*kDim2*Dim],np.float32)
            # Extract training sample features
            for i in range(0,TrnData.shape[3]):
                b=TrnData[:,:,:,i]- OMT_A
                Core_tensor = tl.tenalg.multi_mode_dot(b,Mode_U,list(range(len(rank))),transpose=True)
                x=np.reshape(Core_tensor,(1,kDim1*kDim2*Dim))
                ftr[i,:]=x
            ftr[np.isnan(ftr)] = 0

            # Extract test sample features
            for i in range(0,TestData.shape[3]):
                b=TestData[:,:,:,i]- OMT_A
                Core_tensor = tl.tenalg.multi_mode_dot(b,Mode_U,list(range(len(rank))),transpose=True)
                x=np.reshape(Core_tensor,(1,kDim1*kDim2*Dim))
                fte[i,:]=x
            fte[np.isnan(fte)] = 0
            
            # Seek reconstruction error(MSE和NMSE)
            Unit_Rec_errors=0
            Rec_errors=0
            for i in range(0,TestData.shape[3]):
                OMT_X=TestData[:,:,:,i]
                Core_tensor = tl.tenalg.multi_mode_dot(OMT_X,Mode_U,list(range(len(rank))),transpose=True)
                Temp_tensor = tl.tenalg.multi_mode_dot(Core_tensor,Mode_U,list(range(len(rank))),transpose=False)
                bt_temp=OMT_X-Temp_tensor
                bt7=tl.unfold(bt_temp,mode=1)
                bt8=tl.unfold(OMT_X,mode=1)
                Rec_errors = Rec_errors + np.linalg.norm(bt7,ord='fro')/TestData.shape[3]
                Unit_Rec_errors = Unit_Rec_errors + np.linalg.norm(bt7,ord='fro')/(np.linalg.norm(bt8,ord='fro')*TestData.shape[3])#单个样本重构误差的F范数
            
            print('\n ====== KNN classification ======= \n')
            
            k=1                       # Define the value of the parameter K
            acc=0
            knn=KNeighborsClassifier(n_neighbors=k)
            knn.fit(ftr,TrnLabels)
            lable=knn.predict(fte)
            for i in range(0,len(TestLabels)):
                if lable[i]==TestLabels[i]:
                    acc=acc+1
            Accuracy=acc/len(TestLabels)*100
            print('     Testing Accuracy rate:  %.6f%%'%Accuracy)
            print('\n ====== SVM classification ======= \n')
            PredictLabels=np.zeros(DIR_NUM*test_num,np.uint8)
            classifier=svm.LinearSVC(max_iter=100)
            svm_model= classifier.fit(ftr,TrnLabels)
            acc=0
            cnt_testNum=0
            for i in range(len(TestLabels)):
                cnt_testNum+=1
                pred_label=svm_model.predict(fte[i,:].reshape(1,-1))#han rows of 1, columns unknown
                pred_true=TestLabels[i]
                PredictLabels[i]=pred_label
                if pred_label==pred_true:
                    acc+=1
            Accy=float(acc)/len(TestLabels)*100
            print('===== Results of MPCA, followed by a linear KNN classifier =====\n')
            print('     Reconstruction errors:  %.6f%%'%Unit_Rec_errors)
            print('     SVM Accuracy rate:  %.6f%%'%Accy)
            
            # Open Excel file, create if file does not exist
            try:
                workbook = openpyxl.load_workbook('RMSETD_Errors.xlsx')
            except FileNotFoundError:
                workbook = openpyxl.Workbook()
                workbook.save('RMSETD_Errors.xlsx')

            # Select the first worksheet
            worksheet = workbook.active

            # Record the current time
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Output results
            result='ETH5test'
            result1='Occlusion noise (40%)'
            Exper_dims ='Dimensions'
            result2='SVM_Acuraccy'
            result3='Unit_Reconstruction_errors'
            result4='KNN_Acuraccy'

            # Write results and times to Excel files
            worksheet.append([now, result,result1,Exper_dims,Exper_Num,result2,Accy,result3,Unit_Rec_errors,Rec_errors,result4,Accuracy,mr,Ker_scale,execution_time])

            # Save Excel files
            workbook.save('RMSETD_Errors.xlsx')



 ====== Sample centralization ======= 


 ====== Parameter Definition ======= 


 ====== Find eigenvector ======= 

Objective value: 140.04573938316094

 ====== Iterative updating of the solution ======= 

Objective value: [140.43459681]
Objective value: [140.45784704]
Objective value: [140.45979514]
Objective value: [140.45990959]
程序运行时间为： 4.867939233779907 秒
更新参数完成，进行投影矩阵求解

 ====== Find eigenvector ======= 

Objective value:  151.63%

 ====== Iterative updating of the solution ======= 

Objective value:  151.82%
Objective value:  151.91%
Objective value:  151.95%
Objective value:  151.98%
Objective value:  152.00%
Objective value:  152.02%
Objective value:  152.03%
Objective value:  152.03%
Objective value:  152.04%
Objective value:  152.04%
程序运行时间为： 13.887545347213745 秒

 ====== Extract features ======= 


 ====== KNN classification ======= 

     Testing Accuracy rate:  80.000000%

 ====== SVM classification ======= 

===== Results of MPCA, followed by a linear KNN classifier ===